Visualization 5: Mean CAR Timeliness Analysis (Point Plot)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- Visualization 5: Mean CAR Timeliness Analysis (Point Plot) ---

# Load the aggregated results data
try:
    df_mean = pd.read_csv("./outcome/event_study_mean_results.csv")
except FileNotFoundError:
    print("Error: 'event_study_mean_results.csv' not found in the 'outcome' folder.")
    exit()

# --- Data Preparation ---
# Select a model to analyze across all windows
model_to_plot = 'MM_SPY'
df_plot_data = df_mean[df_mean['Model'] == model_to_plot].reset_index()

# Define the logical order for the x-axis
window_order = ['(-1,+1)', '(-5,+5)', '(-10,+10)']


# --- Plotting ---
# Use catplot with kind='point' to show trends across windows.
# We will create a grid of plots, faceted by Asset.
g = sns.catplot(
    data=df_plot_data,
    x='Window',
    y='MeanCAR',
    hue='EventGroup',
    kind='point',
    col='Asset',
    col_wrap=2,
    order=window_order,
    palette='tab20',
    height=5,
    aspect=1.5,
    sharey=False,  # Let y-axis scale differ for each asset if needed
    legend_out=True
)

# --- Chart Enhancement ---
g.fig.suptitle(f'Mean CAR by Event Window\nModel: {model_to_plot}', y=1.03)
g.set_axis_labels("Event Window", "Mean CAR")
g.set_titles("Asset: {col_name}")
g.despine(left=True)

# Add a horizontal line at y=0
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='black', linewidth=0.8)

g.add_legend(title='Event Group',
             bbox_to_anchor=(1.01, 0.5), # Position legend outside the plot
             loc='upper left')

plt.tight_layout(rect=[0, 0, 0.9, 0.96]) # Adjust layout to make space for legend
plt.show()

Visualization 6: Model Robustness Check (Grouped Bar Chart)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- Visualization 6: Model Robustness Check (Grouped Bar Chart) ---

# Load the aggregated results data
try:
    df_mean = pd.read_csv("./outcome/event_study_mean_results.csv")
except FileNotFoundError:
    print("Error: 'event_study_mean_results.csv' not found in the 'outcome' folder.")
    exit()

# --- Data Preparation ---
# To check robustness, we focus on a specific case: one asset, one window,
# and a few key event groups, while comparing ALL models.
asset_to_plot = 'Bitcoin'
window_to_plot = '(-10,+10)'
groups_to_compare = ['internal_good_train', 'internal_bad_train']

df_plot_data = df_mean[
    (df_mean['Asset'] == asset_to_plot) &
    (df_mean['Window'] == window_to_plot) &
    (df_mean['EventGroup'].isin(groups_to_compare))
].reset_index()


# --- Plotting ---
# We will use a bar chart where the x-axis is the model, faceted by event group.
g = sns.catplot(
    data=df_plot_data,
    x='Model',
    y='MeanCAR',
    kind='bar',
    col='EventGroup',   # Create subplots for 'good' vs 'bad' events
    palette='coolwarm_r',
    height=6,
    aspect=1.2
)

# --- Chart Enhancement ---
g.fig.suptitle(f'Model Robustness Check for {asset_to_plot}\nWindow: {window_to_plot}', y=1.03)
g.set_axis_labels("Factor Model", "Mean CAR")
g.set_titles("Event Group: {col_name}")
g.despine(left=True)

# Add a horizontal line at y=0 and value labels on bars
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='black', linewidth=0.8)
    ax.tick_params(axis='x', rotation=45)
    
    # Add value labels to each bar
    for p in ax.patches:
        ax.annotate(f"{p.get_height():.3f}",
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center',
                    xytext=(0, 9),
                    textcoords='offset points')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

Visualization 7: Daily Cumulative Abnormal Return (CAR) Trend

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- Visualization 7: Daily Cumulative Abnormal Return (CAR) Trend ---

# Load the daily abnormal return data
try:
    df_daily_ar = pd.read_csv("./outcome/event_study_daily_ar.csv")
except FileNotFoundError:
    print("Error: 'event_study_daily_ar.csv' not found in the 'outcome' folder.")
    exit()

# --- Data Preparation ---
# To create a clear comparison, we focus on a specific asset and model.
asset_to_plot = 'Bitcoin'
model_to_plot = 'MM_SPY'

# Filter for the asset and model of interest
df_plot_data = df_daily_ar[
    (df_daily_ar['Asset'] == asset_to_plot) &
    (df_daily_ar['Model'] == model_to_plot)
]

# Step 1: Calculate the average Abnormal Return (AR) for each relative day in each group.
# This gives us the "Average Abnormal Return" (AAR).
df_aar = df_plot_data.groupby(['EventGroup', 'RelativeDay'])['AR'].mean().reset_index()

# Step 2: Calculate the cumulative sum of these daily average returns for each group.
# This gives us the "Average Cumulative Abnormal Return" (ACAR).
df_aar = df_aar.sort_values(by='RelativeDay')
df_aar['ACAR'] = df_aar.groupby('EventGroup')['AR'].cumsum()

# --- Plotting ---
plt.figure(figsize=(14, 8))
sns.lineplot(
    data=df_aar,
    x='RelativeDay',
    y='ACAR',
    hue='EventGroup',
    linewidth=2.5,
    palette='Set1'
)

# --- Chart Enhancement ---
plt.title(f'Average Cumulative Abnormal Return (ACAR) around Event Date\nAsset: {asset_to_plot}, Model: {model_to_plot}', fontsize=16)
plt.xlabel('Days Relative to Event Date (Day 0)', fontsize=12)
plt.ylabel('Average Cumulative Abnormal Return (ACAR)', fontsize=12)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.axvline(0, ls=':', color='black', linewidth=1.5, label='Event Day (0)') # Vertical line for event day
plt.axhline(0, ls=':', color='black', linewidth=1)   # Horizontal line at y=0
plt.legend(title='Event Group')

plt.show()